<a href="https://colab.research.google.com/github/SyedHarshath/Computer-Vision/blob/main/Optical_Flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Optical Flow using Lucas Kanade Algorithm

### Two Frames

In [ ]:
import cv2
import numpy as np

In [ ]:
cap = cv2.VideoCapture("op.mp4")
ret,frame1 = cap.read()
if not ret:
    print("Error: Could not read frame")
    cap.release()
    exit()
ret,frame2 = cap.read()
if not ret:
    print("Error:Could not read frame")
    cap.release()
    exit()

In [ ]:
gray1 = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

In [ ]:
feature_params = dict(maxCorners=400,qualityLevel=0.1,minDistance=7,blockSize=7)
p0 = cv2.goodFeaturesToTrack(gray1,mask=None,**feature_params)

In [ ]:
lk = dict(winSize=(21,21),maxLevel=5,criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,0.03))

In [ ]:
p1,st,err = cv2.calcOpticalFlowPyrLK(gray1,gray2,p0,None,**lk)

In [ ]:
good_new = p1[st==1]
good_old = p0[st==1]

In [ ]:
movement_threshold = 2
for i,(new,old) in enumerate(zip(good_new,good_old)):
    a,b = new.ravel()
    c,d = old.ravel()
    movement_distance = np.sqrt((a - c) ** 2 + (b - d) ** 2)

    if movement_distance < movement_threshold:
        continue

    color = (0, 255, 0)
    if movement_distance > 5:
        color = (0, 0, 255)
    frame2 = cv2.line(frame2,(int(c), int(d)), (int(a), int(b)), color, 2)
    frame2 = cv2.circle(frame2,(int(a), int(b)),5, color, -1)

In [ ]:
cv2.imshow("Optical Flow (Two frames)",frame2)
cv2.waitKey(0)
cv2.destroyAllWindows()
cap.release()

### Whole Video

In [ ]:
cap = cv2.VideoCapture("op.mp4")
ret, old_frame = cap.read()
if not ret:
    print("Error: Could not read video.")
    cap.release()
    exit()

old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
feature_params = dict(maxCorners=400, qualityLevel=0.1, minDistance=7, blockSize=7)
feature_mask = np.ones_like(old_gray, dtype=np.uint8) * 255
feature_mask[:, :150] = 0
p0 = cv2.goodFeaturesToTrack(old_gray, mask=feature_mask, **feature_params)
lk_params = dict(winSize=(21, 21), maxLevel=5,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
mask = np.zeros_like(old_frame)
movement_threshold = 2
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter("optical_flow_output.mp4", fourcc, fps, (width, height))

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    if p1 is None:
        break
    good_new = p1[st == 1]
    good_old = p0[st == 1]
    for new, old in zip(good_new, good_old):
        a, b = new.ravel()
        c, d = old.ravel()
        movement_distance = np.sqrt((a - c) ** 2 + (b - d) ** 2)
        if movement_distance < movement_threshold:
            continue
        color = (255, 0, 0)
        if movement_distance > 5:
            color = (0, 0, 255)
        mask = cv2.line(mask, (int(c), int(d)), (int(a), int(b)), color, 2)
        frame = cv2.circle(frame, (int(a), int(b)), 5, color, -1)
    img = cv2.add(frame, mask)

    cv2.imshow("Optical Flow - Full Video", img)
    out.write(img)
    if cv2.waitKey(30) & 0xFF == 27:
        break
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cap.release()
out.release()
cv2.destroyAllWindows()

### Feature Tracking using KLT

In [ ]:
cap = cv2.VideoCapture('mi.mp4')
feature_params = dict(maxCorners=100,
                      qualityLevel=0.3,
                      minDistance=7,
                      blockSize=7)
lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
color = np.random.randint(0, 255, (100, 3))
ret, old_frame = cap.read()
old_frame = cv2.resize(old_frame, (512, 512))
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
mask = np.zeros_like(old_frame)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.resize(frame, (512, 512))
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    if p1 is not None:
        good_new = p1[st == 1]
        good_old = p0[st == 1]
        new_mask = np.zeros_like(mask)
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            new_mask = cv2.line(new_mask, (int(a), int(b)), (int(c), int(d)), color[i % 100].tolist(), 2)
            frame = cv2.circle(frame, (int(a), int(b)), 5, color[i % 100].tolist(), -1)
        mask = new_mask
        img = cv2.add(frame, mask)

        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)

    else:
        p0 = cv2.goodFeaturesToTrack(frame_gray, mask=None, **feature_params)
        mask = np.zeros_like(frame)

    cv2.imshow('KLT Feature Tracking', img if 'img' in locals() else frame)

    if cv2.waitKey(30) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
